# 5.11 残差网络（ResNet）

让我们先思考一个问题：对神经网络模型添加新的层，充分训练后的模型是否只可能更有效地降低训练误差？理论上，原模型解的空间只是新模型解的空间的子空间。也就是说，如果我们能将新添加的层训练成恒等映射$f(x) = x$，新模型和原模型将同样有效。由于新模型可能得出更优的解来拟合训练数据集，因此添加层似乎更容易降低训练误差。然而在实践中，添加过多的层后训练误差往往不降反升。即使利用批量归一化带来的数值稳定性使训练深层模型更加容易，该问题仍然存在。针对这一问题，何恺明等人提出了残差网络（ResNet） [1]。它在2015年的ImageNet图像识别挑战赛夺魁，并深刻影响了后来的深度神经网络的设计。


## 5.11.1残差块

让我们聚焦于神经网络局部。如图5.9所示，设输入为$\boldsymbol{x}$。假设我们希望学出的理想映射为$f(\boldsymbol{x})$，从而作为图5.9上方激活函数的输入。左图虚线框中的部分需要直接拟合出该映射$f(\boldsymbol{x})$，而右图虚线框中的部分则需要拟合出有关恒等映射的残差映射$f(\boldsymbol{x})-\boldsymbol{x}$。残差映射在实际中往往更容易优化。以本节开头提到的恒等映射作为我们希望学出的理想映射$f(\boldsymbol{x})$。我们只需将图5.9中右图虚线框内上方的加权运算（如仿射）的权重和偏差参数学成0，那么$f(\boldsymbol{x})$即为恒等映射。实际中，当理想映射$f(\boldsymbol{x})$极接近于恒等映射时，残差映射也易于捕捉恒等映射的细微波动。图5.9右图也是ResNet的基础块，即残差块（residual block）。在残差块中，输入可通过跨层的数据线路更快地向前传播。

![设输入为$\boldsymbol{x}$。假设图中最上方激活函数输入的理想映射为$f(\boldsymbol{x})$。左图虚线框中的部分需要直接拟合出该映射$f(\boldsymbol{x})$，而右图虚线框中的部分需要拟合出有关恒等映射的残差映射$f(\boldsymbol{x})-\boldsymbol{x}$](../img/residual-block.svg)

ResNet沿用了VGG全$3\times 3$卷积层的设计。残差块里首先有2个有相同输出通道数的$3\times 3$卷积层。每个卷积层后接一个批量归一化层和ReLU激活函数。然后我们将输入跳过这两个卷积运算后直接加在最后的ReLU激活函数前。这样的设计要求两个卷积层的输出与输入形状一样，从而可以相加。如果想改变通道数，就需要引入一个额外的$1\times 1$卷积层来将输入变换成需要的形状后再做相加运算。

残差块的实现如下。它可以设定输出通道数、是否使用额外的$1\times 1$卷积层来修改通道数以及卷积层的步幅。

### 介绍什么是"残差函数"

1. 残差函数的基本概念
残差（Residual）是指实际值与预测值之间的差异。残差函数就是学习这种差异的函数。在深度学习中，残差网络（ResNet）的核心思想是让网络层学习残差映射，而不是直接学习目标映射。

2. 数学表达
假设我们希望网络学习的映射是H(x)，传统方法是直接让网络拟合H(x)
而残差学习则是：
H(x) = F(x) + x
其中：
- H(x) 是期望的映射
- F(x) 是残差函数
- x 是输入（通过快捷连接/skip connection直接传递）

3. 具体例子
假设我们要训练一个网络来识别数字：

传统方法：
输入：图片数字"2"
目标：直接学习将输入转换为"这是数字2"的整个过程

残差学习方法：
输入：图片数字"2"
残差学习：只需要学习输入与目标之间的差异部分
快捷连接：保留原始输入信息
最终输出：残差部分 + 原始输入

4. 为什么更容易训练？
- 如果最优映射接近于恒等映射，残差网络只需要将F(x)推向零即可
- 快捷连接使得梯度可以直接流向更早的层，缓解了梯度消失问题
- 网络可以选择是否使用残差路径，提供了更大的灵活性

5. 生活中的类比
可以类比为修改文档：
- 传统方法：每次都重写整个文档
- 残差方法：只记录需要修改的部分（残差），保留原文（快捷连接）

这种方法使得网络可以更容易地学习身份映射（identity mapping），特别是在深层网络中，这一特性非常重要，因为它允许信息无损地在网络中传播。这也是为什么ResNet能够训练如此深的网络（超过100层）而不会出现性能下降的重要原因。

In [1]:
import tensorflow as tf
from tensorflow.keras import layers, activations

这是一个"残差块"(Residual Block)的实现，可以把它想象成一个特殊的图像处理模块。它的主要特点是有一条"捷径"，让信息可以跳过主要处理步骤直接传递。

让我用一个生活中的例子来解释：
想象你在一家咖啡店工作，有两种制作咖啡的流程：
1. 主要流程：接单→磨豆→冲泡→加工→成品
2. 快捷通道：直接使用速溶咖啡

这个代码中的设计就类似于把这两种方式结合在一起：
- 主要流程(通过conv1和conv2)：对图像进行复杂的处理和变换
- 快捷通道(X直接相加)：保留原始信息

主要的知识点：

1. **残差连接**
   - 就像咖啡店例子中，除了主要的制作流程，还保留了一条快捷通道
   - 这样做的好处是可以防止信息在传递过程中"变质"太多

2. **1×1卷积的选项**
   - 有时候原始输入和处理后的结果可能不太匹配（就像咖啡浓度不同）
   - 这时候可以用一个小的调整器(1×1卷积)来让它们更匹配

3. **批量归一化**
   - 就像咖啡店要确保每杯咖啡的浓度都适中
   - 这个步骤可以让处理后的数据更稳定

逐层计算`Residual`模块中的可训练参数：

```python
def calculate_params(num_channels, input_channels):
    """
    计算残差块中的参数数量
    
    参数说明：
    - num_channels: 输出通道数
    - input_channels: 输入通道数
    """
    
    # 1. 第一个卷积层 (conv1)
    conv1_params = (3 * 3 * input_channels * num_channels) + num_channels
    print(f"Conv1层参数: {conv1_params}")
    # - 卷积核参数：3×3×input_channels×num_channels
    # - 偏置项参数：num_channels
    
    # 2. 第一个批归一化层 (bn1)
    bn1_params = 4 * num_channels
    print(f"BN1层参数: {bn1_params}")
    # - 4个参数/通道：gamma, beta, running_mean, running_variance
    
    # 3. 第二个卷积层 (conv2)
    conv2_params = (3 * 3 * num_channels * num_channels) + num_channels
    print(f"Conv2层参数: {conv2_params}")
    # - 卷积核参数：3×3×num_channels×num_channels
    # - 偏置项参数：num_channels
    
    # 4. 第二个批归一化层 (bn2)
    bn2_params = 4 * num_channels
    print(f"BN2层参数: {bn2_params}")
    # - 4个参数/通道：gamma, beta, running_mean, running_variance
    
    # 5. 1×1卷积层 (conv3，如果使用)
    if True:  # 假设use_1x1conv=True
        conv3_params = (1 * 1 * input_channels * num_channels) + num_channels
        print(f"Conv3(1×1)层参数: {conv3_params}")
        # - 卷积核参数：1×1×input_channels×num_channels
        # - 偏置项参数：num_channels
    else:
        conv3_params = 0
    
    # 总参数量
    total_params = conv1_params + bn1_params + conv2_params + bn2_params + conv3_params
    print(f"\n总参数量: {total_params}")
    
    return total_params

# 示例：假设输入通道数为64，输出通道数也为64
calculate_params(num_channels=64, input_channels=64)
```

输出示例（当num_channels=64, input_channels=64时）：
```
Conv1层参数: 36928    # (3×3×64×64 + 64)
BN1层参数: 256       # (4×64)
Conv2层参数: 36928    # (3×3×64×64 + 64)
BN2层参数: 256       # (4×64)
Conv3(1×1)层参数: 4160  # (1×1×64×64 + 64)

总参数量: 78528
```

参数计算说明：

1. **卷积层参数**:
   - N×N卷积核：N×N×输入通道数×输出通道数
   - 每个输出通道有一个偏置项
   - 例如3×3卷积：(3×3×64×64) + 64 = 36,928

2. **批归一化层参数**:
   - 每个通道有4个参数：
     * gamma (比例因子)
     * beta (偏移因子)
     * running_mean (运行时均值)
     * running_variance (运行时方差)
   - 总数：4×通道数

3. **1×1卷积层参数**:
   - 参数计算方式与普通卷积层相同
   - 只是卷积核尺寸为1×1

注意事项：
- 实际训练时只有部分参数参与反向传播
- BatchNorm层的running_mean和running_variance在训练时不参与梯度计算
- 参数量会随着通道数的增加而显著增加

这些参数共同构成了模型的可学习参数，影响模型的容量和学习能力。

In [2]:
class Residual(tf.keras.Model):
    """残差块类 - 实现带有跳跃连接的卷积块"""
    
    def __init__(self, num_channels, use_1x1conv=False, strides=1, **kwargs):
        """
        初始化残差块
        参数:
            num_channels: 输出通道数
            use_1x1conv: 是否使用1×1卷积调整维度
            strides: 卷积步长
        """
        super(Residual, self).__init__(**kwargs)
        
        # 第一个卷积层 - 主路径的第一步处理
        self.conv1 = layers.Conv2D(num_channels,
                                   padding='same',
                                   kernel_size=3,
                                   strides=strides)
        
        # 第二个卷积层 - 主路径的第二步处理
        self.conv2 = layers.Conv2D(num_channels, 
                                  kernel_size=3,
                                  padding='same')
        
        # 快捷通道的1×1卷积层(可选) - 用于调整维度匹配
        if use_1x1conv:
            self.conv3 = layers.Conv2D(num_channels,
                                       kernel_size=1,
                                       strides=strides)
        else:
            self.conv3 = None
            
        # 批量归一化层 - 用于稳定数据分布
        self.bn1 = layers.BatchNormalization()
        self.bn2 = layers.BatchNormalization()

    def call(self, X):
        """
        前向传播函数
        
        参数:
            X: 输入数据
        返回:
            经过残差块处理的数据
        """
        # 主路径第一步：卷积+批归一化+激活
        Y = activations.relu(self.bn1(self.conv1(X)))
        
        # 主路径第二步：卷积+批归一化
        Y = self.bn2(self.conv2(Y))
        
        # 快捷通道：如果需要，对输入X进行维度调整
        if self.conv3:
            X = self.conv3(X)
            
        # 主路径输出与快捷通道相加，并进行激活
        return activations.relu(Y + X)

下面我们来查看输入和输出形状一致的情况。

In [3]:
blk = Residual(3)
#tensorflow input shpe     (n_images, x_shape, y_shape, channels).
#mxnet.gluon.nn.conv_layers    (batch_size, in_channels, height, width) 
X = tf.random.uniform((4, 6, 6 , 3))
blk(X).shape

TensorShape([4, 6, 6, 3])

我们也可以在增加输出通道数的同时减半输出的高和宽。

In [4]:
blk = Residual(6, use_1x1conv=True, strides=2)
blk(X).shape

TensorShape([4, 3, 3, 6])

## 5.11.2 ResNet模型

ResNet的前两层跟之前介绍的GoogLeNet中的一样：在输出通道数为64、步幅为2的$7\times 7$卷积层后接步幅为2的$3\times 3$的最大池化层。不同之处在于ResNet每个卷积层后增加的批量归一化层。

In [5]:
net = tf.keras.models.Sequential([
    layers.Conv2D(64, kernel_size=7, strides=2, padding='same'),
    layers.BatchNormalization(), layers.Activation('relu'),
    layers.MaxPool2D(pool_size=3, strides=2, padding='same')
])

一个模块的通道数同输入通道数一致。由于之前已经使用了步幅为2的最大池化层，所以无须减小高和宽。之后的每个模块在第一个残差块里将上一个模块的通道数翻倍，并将高和宽减半。

下面我们来实现这个模块。注意，这里对第一个模块做了特别处理。

In [6]:
class ResnetBlock(tf.keras.layers.Layer):
    def __init__(self,num_channels, num_residuals, first_block=False,**kwargs):
        super(ResnetBlock, self).__init__(**kwargs)
        self.listLayers=[]
        for i in range(num_residuals):
            if i == 0 and not first_block:
                self.listLayers.append(Residual(num_channels, use_1x1conv=True, strides=2))
            else:
                self.listLayers.append(Residual(num_channels))      
    
    def call(self, X):
        for layer in self.listLayers:
            X = layer(X)
        return X

接着我们为ResNet加入所有残差块。这里每个模块使用两个残差块。

### 代码逻辑说明
这段代码定义了一个ResNet神经网络模型，主要包含以下步骤：
1. 初始化阶段设置了一系列层：
   - 初始卷积层
   - 批量归一化层
   - ReLU激活层
   - 最大池化层
   - 4个残差块
   - 全局平均池化层
   - 全连接层

2. 前向传播过程中，数据按顺序通过这些层进行处理。

### 通俗解释
让我用一个工厂生产线的例子来解释这个网络结构：

想象一个生产高级手工艺品的工厂：

1. **初始处理（conv, bn, relu, mp）**
   - 就像原材料先经过初步加工
   - 比如木材需要先切割、打磨、除尘

2. **四个残差块（resnet_block1-4）**
   - 像是4个专业工作站
   - 每个工作站都有"主流程"和"快速通道"
   - 主流程：精细加工
   - 快速通道：保留一些原始特征
   - 例如：
     * 工作站1：基础造型（64个工人）
     * 工作站2：细节雕刻（128个工人）
     * 工作站3：精细打磨（256个工人）
     * 工作站4：最终修饰（512个工人）

3. **最终处理（gap, fc）**
   - 像是最后的质检和包装环节
   - 全局平均池化：综合评估产品质量
   - 全连接层：给产品分类打标签

### 主要知识点
1. **渐进式处理**
   - 特征提取从简单到复杂
   - 工人数量（通道数）逐渐增加：64→128→256→512

2. **残差连接**
   - 保留原始信息的同时进行加工
   - 就像在改进产品时保留原始优点

3. **批量归一化**
   - 确保每道工序的产出稳定
   - 类似于每个工序都有质量标准

4. **模块化设计**
   - 每个残差块都是独立的处理单元
   - 便于管理和调整生产流程


### 代码解释2
1. **初始处理阶段**（前几层）：
```python
self.conv=layers.Conv2D(64, kernel_size=7, strides=2, padding='same')
self.bn=layers.BatchNormalization()
self.relu=layers.Activation('relu')
self.mp=layers.MaxPool2D(pool_size=3, strides=2, padding='same')
```
就像照片刚进入工作室时的基础处理：
- 调整图片大小和基本特征（Conv2D）
- 标准化处理，确保颜色和亮度适中（BatchNormalization）
- 增强重要特征（Activation）
- 保留最显著的特征（MaxPool2D）

2. **深度处理阶段**（ResNet块）：
```python
self.resnet_block1=ResnetBlock(64,num_blocks[0], first_block=True)
self.resnet_block2=ResnetBlock(128,num_blocks[1])
self.resnet_block3=ResnetBlock(256,num_blocks[2])
self.resnet_block4=ResnetBlock(512,num_blocks[3])
```
像是照片的逐层精修：
- 第一层处理基本特征（如轮廓）
- 第二层处理更细节的特征（如纹理）
- 第三层处理更复杂的特征（如物体部分）
- 第四层处理最复杂的特征（如整体结构）

每个块都像是一个专业的处理站，而且采用了"残差连接"的技术，就像保留了原始照片的信息，可以随时参考。

3. **最终处理阶段**：
```python
self.gap=layers.GlobalAvgPool2D()
self.fc=layers.Dense(units=10,activation=tf.keras.activations.softmax)
```
相当于：
- 整合所有处理结果（GlobalAvgPool2D）
- 做出最终分类决定（Dense）

主要知识点：

1. **渐进式处理**
   - 就像专业摄影后期，从基础调整到精细修饰
   - 通道数逐渐增加（64→128→256→512），表示处理的特征越来越丰富

2. **残差结构**
   - 类似于在修图时保留原始版本
   - 可以防止过度处理导致重要信息丢失

3. **金字塔式设计**
   - 从简单到复杂的层次处理
   - 像是专业修图的工作流程，层层递进

4. **参数设置 [2,2,2,2]**
   - 表示每个处理阶段都有2个细化步骤
   - 就像每个修图阶段都经过两轮调整

这就像是一个自动化的专业照片处理工作室，通过一系列精心设计的处理步骤，将输入的图片逐步加工，最终得到想要的结果。每个处理步骤都有其特定的作用，而且彼此之间紧密配合，形成了一个完整的处理流水线。

ResNet每一层的参数数量：

### 1. 初始层参数计算

```python
# 初始卷积层 (7×7, 64通道)
conv_params = 7 * 7 * 1 * 64 + 64  # 卷积核参数 + 偏置
# = 3,136 参数

# 批量归一化层
bn_params = 4 * 64  # 每个通道4个参数(gamma, beta, mean, variance)
# = 256 参数
```

### 2. 残差块参数计算
每个ResNet块包含两个残差单元，每个单元的参数：

**ResBlock1 (64通道)**
```python
# 第一个单元
- Conv3x3: 3 * 3 * 64 * 64 + 64 = 36,928
- BatchNorm: 4 * 64 = 256
- Conv3x3: 3 * 3 * 64 * 64 + 64 = 36,928
- BatchNorm: 4 * 64 = 256
总计：74,368 参数

# 第二个单元（参数相同）
总计：74,368 参数
```

**ResBlock2 (128通道)**
```python
# 第一个单元（包含1x1卷积）
- Conv3x3: 3 * 3 * 64 * 128 + 128 = 73,856
- BatchNorm: 4 * 128 = 512
- Conv3x3: 3 * 3 * 128 * 128 + 128 = 147,584
- BatchNorm: 4 * 128 = 512
- Conv1x1: 1 * 1 * 64 * 128 + 128 = 8,320
总计：230,784 参数

# 第二个单元
- Conv3x3: 3 * 3 * 128 * 128 + 128 = 147,584
- BatchNorm: 4 * 128 = 512
- Conv3x3: 3 * 3 * 128 * 128 + 128 = 147,584
- BatchNorm: 4 * 128 = 512
总计：296,192 参数
```

**ResBlock3 (256通道)**
```python
# 第一个单元（包含1x1卷积）
- Conv3x3: 3 * 3 * 128 * 256 + 256 = 295,168
- BatchNorm: 4 * 256 = 1,024
- Conv3x3: 3 * 3 * 256 * 256 + 256 = 590,080
- BatchNorm: 4 * 256 = 1,024
- Conv1x1: 1 * 1 * 128 * 256 + 256 = 33,024
总计：920,320 参数

# 第二个单元
- Conv3x3: 3 * 3 * 256 * 256 + 256 = 590,080
- BatchNorm: 4 * 256 = 1,024
- Conv3x3: 3 * 3 * 256 * 256 + 256 = 590,080
- BatchNorm: 4 * 256 = 1,024
总计：1,182,208 参数
```

**ResBlock4 (512通道)**
```python
# 第一个单元（包含1x1卷积）
- Conv3x3: 3 * 3 * 256 * 512 + 512 = 1,180,160
- BatchNorm: 4 * 512 = 2,048
- Conv3x3: 3 * 3 * 512 * 512 + 512 = 2,359,808
- BatchNorm: 4 * 512 = 2,048
- Conv1x1: 1 * 1 * 256 * 512 + 512 = 131,584
总计：3,675,648 参数

# 第二个单元
- Conv3x3: 3 * 3 * 512 * 512 + 512 = 2,359,808
- BatchNorm: 4 * 512 = 2,048
- Conv3x3: 3 * 3 * 512 * 512 + 512 = 2,359,808
- BatchNorm: 4 * 512 = 2,048
总计：4,723,712 参数
```

### 3. 最终层参数计算
```python
# 全连接层
fc_params = 512 * 10 + 10  # 输入维度 * 输出维度 + 偏置
# = 5,130 参数
```

### 总参数量汇总
```python
总参数 = 初始层参数 + 所有残差块参数 + 最终层参数
= (3,392) + (148,736 + 526,976 + 2,102,528 + 8,399,360) + 5,130
≈ 11,186,122 参数
```

注意事项：
1. 批量归一化层的running_mean和running_variance在训练时不参与梯度更新
2. 参数量随着网络深度和通道数的增加而显著增加
3. 1×1卷积虽然看起来简单，但在通道数较大时也会带来可观的参数量

这些参数共同构成了模型的可学习参数，影响模型的容量和学习能力。

In [7]:
class ResNet(tf.keras.Model):
    def __init__(self,num_blocks,**kwargs):
        super(ResNet, self).__init__(**kwargs)
        self.conv=layers.Conv2D(64, kernel_size=7, strides=2, padding='same')
        self.bn=layers.BatchNormalization()
        self.relu=layers.Activation('relu')
        self.mp=layers.MaxPool2D(pool_size=3, strides=2, padding='same')
        self.resnet_block1=ResnetBlock(64,num_blocks[0], first_block=True)
        self.resnet_block2=ResnetBlock(128,num_blocks[1])
        self.resnet_block3=ResnetBlock(256,num_blocks[2])
        self.resnet_block4=ResnetBlock(512,num_blocks[3])
        self.gap=layers.GlobalAvgPool2D()
        self.fc=layers.Dense(units=10,activation=tf.keras.activations.softmax)

    def call(self, x):
        x=self.conv(x)
        x=self.bn(x)
        x=self.relu(x)
        x=self.mp(x)
        x=self.resnet_block1(x)
        x=self.resnet_block2(x)
        x=self.resnet_block3(x)
        x=self.resnet_block4(x)
        x=self.gap(x)
        x=self.fc(x)
        return x
    
mynet=ResNet([2,2,2,2])

最后，与GoogLeNet一样，加入全局平均池化层后接上全连接层输出。

这里每个模块里有4个卷积层（不计算 1×1卷积层），加上最开始的卷积层和最后的全连接层，共计18层。这个模型通常也被称为ResNet-18。通过配置不同的通道数和模块里的残差块数可以得到不同的ResNet模型，例如更深的含152层的ResNet-152。虽然ResNet的主体架构跟GoogLeNet的类似，但ResNet结构更简单，修改也更方便。这些因素都导致了ResNet迅速被广泛使用。
在训练ResNet之前，我们来观察一下输入形状在ResNet不同模块之间的变化。

In [8]:
X = tf.random.uniform(shape=(1,  224, 224 , 1))
for layer in mynet.layers:
    X = layer(X)
    print(layer.name, 'output shape:\t', X.shape)

conv2d_6 output shape:	 (1, 112, 112, 64)
batch_normalization_5 output shape:	 (1, 112, 112, 64)
activation_1 output shape:	 (1, 112, 112, 64)
max_pooling2d_1 output shape:	 (1, 56, 56, 64)
resnet_block output shape:	 (1, 56, 56, 64)
resnet_block_1 output shape:	 (1, 28, 28, 128)
resnet_block_2 output shape:	 (1, 14, 14, 256)
resnet_block_3 output shape:	 (1, 7, 7, 512)
global_average_pooling2d output shape:	 (1, 512)
dense output shape:	 (1, 10)


## 5.11.3  获取数据和训练模型

下面我们在Fashion-MNIST数据集上训练ResNet。

In [9]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()
x_train = x_train.reshape((60000, 28, 28, 1)).astype('float32') / 255
x_test = x_test.reshape((10000, 28, 28, 1)).astype('float32') / 255

mynet.compile(loss='sparse_categorical_crossentropy',
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['accuracy'])

history = mynet.fit(x_train, y_train,
                    batch_size=64,
                    epochs=5,
                    validation_split=0.2)
test_scores = mynet.evaluate(x_test, y_test, verbose=2)
# 保存模型权重
model_weights_path = 'resnet_model.weights.h5'
mynet.save_weights(model_weights_path)
print(f'model weights saved to: {model_weights_path}')



29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/5


I0000 00:00:1729589537.490081      65 service.cc:145] XLA service 0x7eb1940045c0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1729589537.490136      65 service.cc:153]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0


 10/750 ━━━━━━━━━━━━━━━━━━━━ 9s 12ms/step - accuracy: 0.1699 - loss: 2.2498

I0000 00:00:1729589543.347202      65 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'loop_add_subtract_fusion_28', 4 bytes spill stores, 12 bytes spill loads
ptxas warning : Registers are spilled to local memory in function 'loop_add_subtract_fusion_2', 4 bytes spill stores, 12 bytes spill loads

I0000 00:00:1729589543.360139      65 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


750/750 ━━━━━━━━━━━━━━━━━━━━ 26s 15ms/step - accuracy: 0.6826 - loss: 0.8170 - val_accuracy: 0.8637 - val_loss: 0.3834
Epoch 2/5
750/750 ━━━━━━━━━━━━━━━━━━━━ 9s 12ms/step - accuracy: 0.8674 - loss: 0.3624 - val_accuracy: 0.8739 - val_loss: 0.3487
Epoch 3/5
750/750 ━━━━━━━━━━━━━━━━━━━━ 9s 12ms/step - accuracy: 0.8836 - loss: 0.3079 - val_accuracy: 0.8898 - val_loss: 0.3099
Epoch 4/5
750/750 ━━━━━━━━━━━━━━━━━━━━ 9s 12ms/step - accuracy: 0.8988 - loss: 0.2695 - val_accuracy: 0.8851 - val_loss: 0.3146
Epoch 5/5
750/750 ━━━━━━━━━━━━━━━━━━━━ 9s 12ms/step - accuracy: 0.9075 - loss: 0.2455 - val_accuracy: 0.8960 - val_loss: 0.2787
313/313 - 2s - 6ms/step - accuracy: 0.8955 - loss: 0.2905
model weights saved to: resnet_model.weights.h5


## 小结

* 残差块通过跨层的数据通道从而能够训练出有效的深度神经网络。
* ResNet深刻影响了后来的深度神经网络的设计。


## 练习

* 参考ResNet论文的表1来实现不同版本的ResNet [1]。
* 对于比较深的网络， ResNet论文中介绍了一个“瓶颈”架构来降低模型复杂度。尝试实现它 [1]。
* 在ResNet的后续版本里，作者将残差块里的“卷积、批量归一化和激活”结构改成了“批量归一化、激活和卷积”，实现这个改进（[2]，图1）。



## 参考文献

[1] He, K., Zhang, X., Ren, S., & Sun, J. (2016). Deep residual learning for image recognition. In Proceedings of the IEEE conference on computer vision and pattern recognition (pp. 770-778).

[2] He, K., Zhang, X., Ren, S., & Sun, J. (2016, October). Identity mappings in deep residual networks. In European Conference on Computer Vision (pp. 630-645). Springer, Cham.

